In [63]:
import pymysql
from yaml import load, FullLoader
import pandas as pd
import time
from datetime import datetime
import numpy as np
from tqdm import tqdm

In [3]:
def load_config(config_path:str):
    with open(config_path, "r") as f:
        config = load(f, FullLoader)
    return config

In [4]:
sset_contract_address = "0x34d85c9cdeb23fa97cb08333b511ac86e1c4e258"
rec_10 = []

config = load_config(config_path="dev_config.yaml")

# mysql 연결
db_name = config['gcp_db']['database']
conn = pymysql.connect(host=config['gcp_db']['host'], user=config['gcp_db']['user'], password=config['gcp_db']['password'], db=db_name, port=config['gcp_db']['port'], charset='utf8')
cursor = conn.cursor()

cmd = f"SELECT {db_name}.ITEM.token_id, {db_name}.ITEM.predict_value, {db_name}.PRICE_OFFER.Seller_price, {db_name}.PRICE_OFFER.Seller_price_type, {db_name}.PRICE_OFFER.Buyer_price, {db_name}.PRICE_OFFER.Buyer_price_type " \
        + f"from {db_name}.ITEM left join {db_name}.PRICE_OFFER on {db_name}.ITEM.token_id={db_name}.PRICE_OFFER.Token_ID"
        
cursor.execute(cmd)
data=cursor.fetchall() 
predict_price = pd.DataFrame(data)

# #diff_list = list()
# for i in tqdm(range(len(predict_price))):
#     token_id, predict_value, seller_price, buyer_price = predict_price.loc[i, :]
#     rec_10.put((-(predict_value-max(seller_price, buyer_price)), predict_value, token_id))

In [25]:
# apecoin , dai stablecoin 제거
# 3은 Seller_price_type, 5는 Buyer_price_type
predict_price.loc[predict_price[3]=='ApeCoin',2]=np.nan 
predict_price.loc[predict_price[5]=='ApeCoin',4]=np.nan
predict_price.loc[predict_price[5]=='Dai Stablecoin',4]=np.nan

In [50]:
# 둘다 nan인 경우 제거
eth_predict_price = predict_price[np.logical_not(np.logical_and(predict_price[2].isna(),predict_price[4].isna()))].loc[:,[0,1,2,4]]

In [57]:
# 비교를 위해 nan값 0으로
eth_predict_price=eth_predict_price.fillna(0)

In [67]:
# 가격 차이 가장 큰거 10개
rec10=eth_predict_price.iloc[np.argpartition(eth_predict_price[1]-np.maximum(eth_predict_price[2],eth_predict_price[4]),-10)[-10:],:]

In [68]:
# 이런식으로 만들어짐
rec10

,0,1,2,4
7240,7240,19.3555,0.0,3.0899
3438,3438,18.3867,0.0,2.0860
6719,6719,19.5120,0.0,3.0899
5322,5322,18.6192,0.0,2.1313
3822,3822,19.6094,0.0,3.0990
45138,45138,16.5638,0.0,0.0270
2255,2255,19.6798,0.0,3.0990
4788,4788,19.7959,0.0,3.0990
3938,3938,19.2028,0.0,1.8525
6373,6373,19.7507,0.0,2.1233
